In [1]:
import re
import time
import requests
import random
from bs4 import BeautifulSoup
import openpyxl
from openpyxl import workbook, load_workbook
from openpyxl.utils import get_column_letter

In [11]:
# A list of user-agent strings to choose from
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59 Safari/537.36 Edg/91.0.864.59",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0.2",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0.2",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 11; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.120 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 11; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.120 Mobile Safari/537.36 EdgA/46.03.4.5155",
    "Mozilla/5.0 (Linux; Android 11; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0.2",
]

#Access Excel sheet data
wb = load_workbook('PC_component_list.xlsx')

#Specify scraping time delay
min_delay = 0
max_delay = 1

In [ ]:
#Run consecutive functions to pull 9 pages of product URLs & fill an empty list
product_url_list = []

def pull_product_url_page_1():
    #Use URL of page 1
    url = 'https://geizhals.at/?cat=mbp4_1200#productlist'
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    div_elements = soup.find_all('div', class_='cell productlist__item productlist__name')

    for div in div_elements:
        a_element = div.find('a')
        if a_element and 'href' in a_element.attrs:
            href_value = a_element['href']
            product_url_list.append(f"https://geizhals.at/{href_value}")

def pull_product_url(page_num, max_pages):
    if page_num > max_pages:
        return
    #Use URL of page 2, as {page_num} can then be dynamic
    base_url = 'https://geizhals.at/?cat=mbp4_1200&pg={page_num}#productlist'
    url = base_url.format(page_num=page_num)
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    div_elements = soup.find_all('div', class_='cell productlist__item productlist__name')

    for div in div_elements:
        a_element = div.find('a')
        if a_element and 'href' in a_element.attrs:
            href_value = a_element['href']
            product_url_list.append(f"https://geizhals.at/{href_value}")

    next_page_num = page_num + 1
    delay = random.uniform(min_delay, max_delay)
    time.sleep(delay)
    pull_product_url(next_page_num, max_pages)


pull_product_url_page_1()
pull_product_url(2, max_pages=4)
print(product_url_list)


In [16]:
#Fill Excel database with product URLs

ws = wb['NEW']
for index, url in enumerate(product_url_list, start=2):
    cell = ws.cell(row=index, column=2, value=url)
wb.save('PC_component_list_updated.xlsx')

In [28]:
#Create a function to scrape gpu data

def gpu_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = data[0].text
    memory = data[1].text
    dimensions = data[12].text
    connections = data[7].text
    first_release = data[29].text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    return title, component_price, image, memory, dimensions, connections, first_release

In [ ]:
#Create a function to scrape cpu data

def cpu_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    soup.find_all('meta')
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    graphics = soup.find_all('dd')[5].text
    base_clock = soup.find_all('dd')[3].text
    boost_clock = soup.find_all('dd')[2].text
    socket = soup.find_all('dd')[6].text
    ddr = soup.find_all('dd')[13].text
    pcie = soup.find_all('dd')[21].text
    introduced = soup.find_all('dd')[23].text
    return component_price, image, graphics, base_clock, boost_clock, socket, ddr, pcie, introduced

In [20]:
#Create a function to scrape mobo data

def mobo_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = soup.find('h1').text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    form_factor = data[0].text
    socket = data[1].text
    chipset = data[2].text
    ram = data[4].text
    internal_headers = data[7].text
    buttons = data[10].text
    audio = data[11].text
    graphics = data[12].text
    wireless = data[13].text

    return title, component_price, image, form_factor, socket, chipset, ram, internal_headers, buttons, audio, graphics, wireless

In [ ]:
#Create a function to scrape RAM data

def ram_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = soup.find('h1').text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    ddr = data[0].text
    rate = data[1].text
    module = data[2].text
    rgb = data[11].text

    return title, component_price, image, ddr, rate, module, rgb

In [ ]:
#Create a function to scrape SSD data

def ssd_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = soup.find('h1').text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    form_factor = data[1].text
    interface = data[2].text
    read = data[3].text
    write = data[4].text

    return title, component_price, image, form_factor, interface, read, write

In [7]:
#Create a function to scrape House data

def house_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = soup.find('h1').text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    front_io = data[2].text
    front_fan = data[4].text
    rear_fan = data[5].text
    top_fan = data[8].text
    radiator = data[11].text
    mobo = data[13].text
    psu = data[14].text
    cpu_cooler = data[16].text
    gpu_length = data[17].text
    colour = data[18].text
    rgb = data[19].text
    description = data[31].text

    return title, component_price, image, front_io, front_fan, rear_fan, top_fan, radiator, mobo, psu, cpu_cooler, gpu_length, colour, rgb, description

In [3]:
#Create a function to scrape AIO data

def aio_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = soup.find('h1').text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    heatsink = data[2].text
    socket = data[3].text
    length = data[4].text
    thickness = data[5].text
    fans = data[7].text
    pump = data[9].text

    return title, component_price, image, heatsink, socket, length, thickness, fans, pump

In [14]:
#Create a function to scrape CPU fan data

def cpu_fan_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        html = response.text
        soup = BeautifulSoup(html,'html.parser')

        title = soup.find('h1').text
        component_price = soup.find_all('meta')[3]['content']
        image = soup.find_all('meta')[2]['content']

        data = soup.find_all('dd')
        dimensions = data[1].text if len(data) > 1 else "N/A"
        socket_amd = data[5].text if len(data) > 5 else "N/A"
        socket_intel = data[6].text if len(data) > 6 else "N/A"
        wattage = data[7].text if len(data) > 7 else "N/A"
        rgb = data[8].text if len(data) > 8 else "N/A"
        extra = data[9].text if len(data) > 9 else "N/A"

        return title, component_price, image, dimensions, socket_amd, socket_intel, wattage, rgb, extra
    
    except requests.exceptions.RequestException as e:
        print(f"Error while fetching {url}: {e}")
        return None, None, None, None, None, None, None, None, None


In [ ]:
#Create a function to scrape PSU data

def psu_data_gather(url):
    user_agent = random.choice(user_agents)
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find_all('dd')
    title = soup.find('h1').text
    component_price = soup.find_all('meta')[3]['content']
    image = soup.find_all('meta')[2]['content']
    one = data[1].text
    two = data[2].text
    three = data[3].text
    four = data[4].text
    five = data[5].text
    six = data[6].text
    seven = data[7].text
    eight = data[8].text
    nine = data[9].text
    ten = data[10].text
    eleven = data[11].text
    twelve = data[12].text
    thirteen = data[13].text
    fourteen = data[14].text
    fifteen = data[15].text
    sixteen = data[16].text
    seventeen = data[17].text
    eighteen = data[18].text

    return one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirteen, fourteen, fifteen, sixteen, seventeen, eighteen

In [ ]:
#Accessing GPU Excel sheet data

def update_gpu_data():
    ws = wb['GPU']
    for row in ws.iter_rows(min_row=2, max_row=451, min_col=1, max_col=8):
        component_url = row[1].value
        title, component_price, image, memory, dimensions, connections, first_release = gpu_data_gather(component_url)
        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = memory
        row[4].value = dimensions
        row[5].value = connections
        row[6].value = first_release
        row[7].value = image
        print(title, first_release)

update_gpu_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_cpu_data():
    ws = wb['CPU']
    for row in ws.iter_rows(min_row=2, max_row=70, min_col=1, max_col=11):
        component_name = row[0].value
        component_url = row[1].value
        component_price, image, graphics, base_clock, boost_clock, socket, ddr, pcie, introduced = cpu_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[2].value = component_price
        row[3].value = socket
        row[4].value = graphics
        row[5].value = base_clock
        row[6].value = boost_clock
        row[7].value = pcie
        row[8].value = ddr
        row[9].value = introduced
        row[10].value = image
        print(component_name, component_price, base_clock, socket)

update_cpu_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_mobo_data():
    ws = wb['MOBO']
    for row in ws.iter_rows(min_row=2, max_row=340, min_col=1, max_col=13):
        component_url = row[1].value
        title, component_price, image, form_factor, socket, chipset, ram, internal_headers, buttons, audio, graphics, wireless = mobo_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = socket
        row[4].value = chipset
        row[5].value = ram
        row[6].value = form_factor
        row[7].value = internal_headers
        row[8].value = buttons
        row[9].value = audio
        row[10].value = graphics
        row[11].value = wireless
        row[12].value = image
        print(title, component_price, socket, wireless)

update_mobo_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_ram_data():
    ws = wb['RAM']
    for row in ws.iter_rows(min_row=2, max_row=68, min_col=1, max_col=8):
        component_url = row[1].value
        title, component_price, image, ddr, rate, module, rgb = ram_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = ddr
        row[4].value = rate
        row[5].value = module
        row[6].value = rgb
        row[7].value = image
        print(title)
        print(rate)

update_ram_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_ssd_data():
    ws = wb['SSD']
    for row in ws.iter_rows(min_row=2, max_row=51, min_col=1, max_col=9):
        component_url = row[1].value
        title, component_price, image, form_factor, interface, read, write = ssd_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = form_factor
        row[4].value = interface
        row[6].value = read
        row[7].value = write
        row[8].value = image
        
        print(title)
        print(form_factor)
        print(interface)
        print(read)
        print(write)

update_ssd_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_psu_data():
    ws = wb['PSU']
    for row in ws.iter_rows(min_row=2, max_row=78, min_col=1, max_col=30):
        component_url = row[1].value
        one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirteen, fourteen, fifteen, sixteen, seventeen, eighteen = psu_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[12].value = one
        row[13].value = two
        row[14].value = three
        row[15].value = four
        row[16].value = five
        row[17].value = six
        row[18].value = seven
        row[19].value = eight
        row[20].value = nine
        row[21].value = ten
        row[22].value = eleven
        row[23].value = twelve
        row[24].value = thirteen
        row[25].value = fourteen
        row[26].value = fifteen
        row[27].value = sixteen
        row[28].value = seventeen
        row[29].value = eighteen

update_psu_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_aio_data():
    ws = wb['AIO']
    for row in ws.iter_rows(min_row=2, max_row=421, min_col=1, max_col=10):
        component_url = row[1].value
        title, component_price, image, heatsink, socket, length, thickness, fans, pump = aio_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = heatsink
        row[4].value = socket
        row[5].value = length
        row[6].value = thickness
        row[7].value = fans
        row[8].value = pump
        row[9].value = image
        
        print(title)
        print(heatsink)
        print(length)
        print(pump)

update_aio_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_house_data():
    ws = wb['House']
    for row in ws.iter_rows(min_row=2, max_row=151, min_col=1, max_col=16):
        component_url = row[1].value
        title, component_price, image, front_io, front_fan, rear_fan, top_fan, radiator, mobo, psu, cpu_cooler, gpu_length, colour, rgb, description = house_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = front_io
        row[4].value = front_fan
        row[5].value = rear_fan
        row[6].value = top_fan
        row[7].value = radiator
        row[8].value = mobo
        row[9].value = psu
        row[10].value = cpu_cooler
        row[11].value = gpu_length
        row[12].value = colour
        row[13].value = rgb
        row[14].value = description
        row[15].value = image
        
        print(title)
        print(mobo)
        print(colour)
        print(description)

update_house_data()
wb.save('PC_component_list_updated.xlsx')

In [ ]:
def update_cpu_fan_data():
    ws = wb['CPU_fan']
    for row in ws.iter_rows(min_row=2, max_row=241, min_col=1, max_col=10):
        component_url = row[1].value
        title, component_price, image, dimensions, socket_amd, socket_intel, wattage, rgb, extra = cpu_fan_data_gather(component_url)

        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        row[0].value = title
        row[2].value = component_price
        row[3].value = dimensions
        row[4].value = socket_amd
        row[5].value = socket_intel
        row[6].value = wattage
        row[7].value = rgb
        row[8].value = image
        row[9].value = extra
        
        print(title)
        print(dimensions)
        print(wattage)

update_cpu_fan_data()
wb.save('PC_component_list_updated.xlsx')